[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/text-to-video-synthesis-colab/blob/main/text_to_video_synthesis.ipynb)

In [ ]:
!git clone https://huggingface.co/camenduru/text-to-video-synthesis /root/.cache/modelscope
!pip install -q git+https://github.com/camenduru/modelscope open_clip_torch pytorch_lightning

In [ ]:
from modelscope.pipelines import pipeline
from modelscope.outputs import OutputKeys

pipe = pipeline('text-to-video-synthesis', 'damo/text-to-video-synthesis')
test_text = {
        'text': 'A panda eating bamboo on a rock.',
    }
output_video_path = pipe(test_text,)[OutputKeys.OUTPUT_VIDEO]
print('output_video_path:', output_video_path)

from IPython.display import HTML
from base64 import b64encode

mp4 = open(output_video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)